In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
cd ../src/

In [ ]:
# !pip install -qU python-gdcm pydicom pylibjpeg

In [ ]:
import os
import cv2
import glob
import gdcm
import pydicom
import zipfile
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from collections import Counter
from tqdm.notebook import tqdm
from joblib import Parallel, delayed

In [ ]:
DATA_PATH = "../input/train_images/"

print('Number of training patients :', len(os.listdir(DATA_PATH)))

In [ ]:
df = pd.read_csv('../input/train_series_descriptions.csv')
df["orient"] = df["series_description"].apply(lambda x: x.split()[0])
df["weighting"] = df["series_description"].apply(lambda x: x.split()[1])

In [ ]:
SAVE_FOLDER = "../input/npy2/"
os.makedirs(SAVE_FOLDER, exist_ok=True)

In [ ]:
from inference.processing import process_2

def process_and_save(study, series, orient, data_path, save_folder=""):
    imgs, df_series = process_2(int(study), int(series), orient, data_path=data_path)
    
    if save_folder:
        np.save(save_folder + f"{study}_{series}.npy", imgs)

    return {
        "study_id": study,
        "series_id": series,
        "frames": df_series.instance_number.values.tolist(),
    }

In [ ]:
offsets = Parallel(n_jobs=16)(
    delayed(process_and_save)(
        df['study_id'][i],
        df['series_id'][i],
        df['orient'][i],
        DATA_PATH,
        SAVE_FOLDER
    ) for i in tqdm(range(len(df)))
)

In [ ]:
df_frames = pd.DataFrame(offsets)
df_frames.to_csv('../input/df_frames.csv', index=False)

In [ ]:
for i in range(3):
    study = df['study_id'][i]
    series = df['series_id'][i]

    imgs = np.load(SAVE_FOLDER + f"{study}_{series}.npy")

    for i in range(len(imgs)):
        img = imgs[i]
        if not (i % (len(imgs) // 5 + 1)):
        # if i > 40:
            plt.figure(figsize=(5, 5))
            plt.imshow(img, cmap="gray")
            plt.title(
                f"Study {study} - Series {series} - Frame {i}/{len(imgs)} - Shape {img.shape}"
            )
            plt.show()

    # break

    break

Done ! 